***

# **STA 221 Project Code**

***

In [91]:
# packages

import time
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from tweety import Twitter
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
import joblib
import pickle

In [6]:
# Loading dataset, this is our main frame

tweets = pd.read_csv("Tweets.csv")

In [6]:
tweets.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

train_vectors = vectorizer.fit_transform(tweets['text'])


In [5]:
# SVM | https://medium.com/scrapehero/sentiment-analysis-using-svm-338d418e3ff1

import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, tweets['airline_sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testData['Label'], prediction_linear, output_dict=True)
print('positive: ', report['pos'])
print('negative: ', report['neg'])

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [7]:
# https://www.educative.io/answers/how-to-use-svm-for-sentiment-analysis

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report


text = tweets['text'].values
labels = tweets['airline_sentiment'].values

text_train, text_test, labels_train, labels_test = train_test_split(text, labels, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
features_train = vectorizer.fit_transform(text_train)
features_test = vectorizer.transform(text_test)

svm = SVC(kernel='linear')
svm.fit(features_train, labels_train)

predictions = svm.predict(features_test)
new_text = ["I hate Devin Booker", "Thai Canteen has awful service.", "The new Titebond glue is amazing."]
new_features = vectorizer.transform(new_text)
new_predictions = svm.predict(new_features)
print(new_predictions)

# Step 7: Generate the classification report to evaluate the model
print(classification_report(labels_test, predictions))

['negative' 'neutral' 'positive']
              precision    recall  f1-score   support

    negative       0.86      0.87      0.86      1889
     neutral       0.58      0.56      0.57       580
    positive       0.70      0.71      0.71       459

    accuracy                           0.78      2928
   macro avg       0.71      0.71      0.71      2928
weighted avg       0.78      0.78      0.78      2928



In [8]:

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

text = tweets['text'].values
labels = tweets['airline_sentiment'].values

text_train, text_test, labels_train, labels_test = train_test_split(text, labels, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(text_train)
X_test_tfidf = vectorizer.transform(text_test)

# Hyperparameter tuning using Grid Search
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
svm_model = SVC()
grid_search = GridSearchCV(svm_model, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_tfidf, labels_train)

print("Best Parameters:", grid_search.best_params_)

# Training on the params chosen
best_svm_model = grid_search.best_estimator_
best_svm_model.fit(X_train_tfidf, labels_train)

y_pred = best_svm_model.predict(X_test_tfidf)

accuracy = accuracy_score(labels_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print('\nClassification Report:')
print(classification_report(labels_test, y_pred))

Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.80

Classification Report:
              precision    recall  f1-score   support

    negative       0.84      0.91      0.88      1889
     neutral       0.63      0.56      0.59       580
    positive       0.77      0.63      0.69       459

    accuracy                           0.80      2928
   macro avg       0.75      0.70      0.72      2928
weighted avg       0.79      0.80      0.79      2928



Using a base model and a hypertuned parameter model, we see similar results. Let's try and subset to see if we can improve our accuracy in both reading and categorizing.

In [48]:
# Cleaning the text

def tweet_to_words(tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words ))

nltk.download('stopwords')
tweets['cleaned_text']=tweets['text'].apply(lambda x: tweet_to_words(x))

[nltk_data] Downloading package stopwords to /Users/fish/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
# prep for next

le = LabelEncoder()

tweets_noneg = tweets[tweets['airline_sentiment'] != 'neutral']

le.fit(tweets_noneg['airline_sentiment'])

tweets_noneg['sent_encode'] = le.transform(tweets_noneg['airline_sentiment'])

/var/folders/9k/rc_30bp10dx944lx6yjz_m0w0000gn/T/ipykernel_4081/3957936042.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_noneg['sent_encode'] = le.transform(tweets_noneg['airline_sentiment'])


In [106]:
# Subsetting to only use tweets with Confidence level above .7 and no neutral reviews

data_clean = tweets_noneg[tweets_noneg['airline_sentiment_confidence'] > 0.7]

text = data_clean['cleaned_text'].values
labels = data_clean['airline_sentiment'].values # sent_encode

text_train, text_test, labels_train, labels_test = train_test_split(text, labels, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(text_train)
X_test_tfidf = vectorizer.transform(text_test)


joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

# Hyperparameter tuning using Grid Search
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
svm_model = SVC()
grid_search = GridSearchCV(svm_model, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_tfidf, labels_train)

print("Best Parameters:", grid_search.best_params_)

# Training on the params chosen
best_svm_model = grid_search.best_estimator_
best_svm_model.fit(X_train_tfidf, labels_train)

y_pred = best_svm_model.predict(X_test_tfidf)

accuracy = accuracy_score(labels_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print('\nClassification Report:')
print(classification_report(labels_test, y_pred))

Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.94

Classification Report:
              precision    recall  f1-score   support

    negative       0.95      0.98      0.97      1505
    positive       0.90      0.78      0.83       321

    accuracy                           0.94      1826
   macro avg       0.92      0.88      0.90      1826
weighted avg       0.94      0.94      0.94      1826



In [99]:
# What if we look at just no neutral reviews, not subsetting for confidence as well

text = tweets_noneg['cleaned_text'].values
labels = tweets_noneg['airline_sentiment'].values

text_train, text_test, labels_train, labels_test = train_test_split(text, labels, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(text_train)
X_test_tfidf = vectorizer.transform(text_test)

# Hyperparameter tuning using Grid Search
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
svm_model = SVC()
grid_search = GridSearchCV(svm_model, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_tfidf, labels_train)

print("Best Parameters:", grid_search.best_params_)

# Training on the params chosen
best_svm_model = grid_search.best_estimator_
best_svm_model.fit(X_train_tfidf, labels_train)

y_pred = best_svm_model.predict(X_test_tfidf)

accuracy = accuracy_score(labels_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print('\nClassification Report:')
print(classification_report(labels_test, y_pred))

Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.91

Classification Report:
              precision    recall  f1-score   support

    negative       0.93      0.97      0.95      1862
    positive       0.83      0.70      0.76       447

    accuracy                           0.91      2309
   macro avg       0.88      0.83      0.85      2309
weighted avg       0.91      0.91      0.91      2309



In [107]:
# Testing some tweets pulled from my friend's twitter @7weekslive

# https://twitter.com/lithub/status/1729252483594342681

# https://twitter.com/JhariahClare/status/1730093772220879058

# https://twitter.com/RamiJarrah/status/1729611779787964717

# https://twitter.com/sitcomabed/status/1730753622684623083

# https://twitter.com/muttnik/status/1731119902575178202

# From the tweets above
test_sentences = ["Gaza’s main public library has been destroyed.",
                  "Palestine is still persevering. They will be free. Do not let them do it alone please.",
                   "Bernie Sanders is now the lone senator from Vermont, NOT calling for a ceasefire.",
                   "im feeling really grateful for every club on my campus that has been using their time to help fight for Palestinians.  A lot of those clubs are mostly comprised of poc and it just solidifies the solidarity between us. Our president is so horrible so its nice to have their support.",
                   "Rain or shine SF shows up for Palestine",
                   "The 49ers are winning the super bowl this year."]

# This was the model stored from the best performing accuracy test using SVM. We did this one with no neutral reviews and greater than 70% confidence
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# prepocessing using function from above
cleaned_test_sentences = [tweet_to_words(sentence) for sentence in test_sentences]

X_test_sentences_tfidf = vectorizer.transform(cleaned_test_sentences)

predictions = best_svm_model.predict(X_test_sentences_tfidf)

# Printing
for sentence, prediction in zip(test_sentences, predictions):
    print(f"Sentence: {sentence}")
    print(f"Predicted Sentiment: {prediction}")
    print("------")


Sentence: Gaza’s main public library has been destroyed.
Predicted Sentiment: negative
------
Sentence: Palestine is still persevering. They will be free. Do not let them do it alone please.
Predicted Sentiment: negative
------
Sentence: Bernie Sanders is now the lone senator from Vermont, NOT calling for a ceasefire.
Predicted Sentiment: negative
------
Sentence: im feeling really grateful for every club on my campus that has been using their time to help fight for Palestinians.  A lot of those clubs are mostly comprised of poc and it just solidifies the solidarity between us. Our president is so horrible so its nice to have their support.
Predicted Sentiment: negative
------
Sentence: Rain or shine SF shows up for Palestine
Predicted Sentiment: negative
------
Sentence: The 49ers are winning the super bowl this year.
Predicted Sentiment: negative
------


In [54]:
cm = confusion_matrix(labels_test, y_pred)
cm

array([[1801,   61],
       [ 137,  310]])

In [59]:
import plotly.figure_factory as ff


# Create confusion matrix figure using Plotly
fig = ff.create_annotated_heatmap(
    z=cm,
    x=['Predicted 0', 'Predicted 1'],
    y=['Actual 0', 'Actual 1'],
    colorscale='Viridis'
)

# Add custom annotations
fig.update_layout(
    title_text='Confusion Matrix',
    xaxis=dict(title='Predicted'),
    yaxis=dict(title='Actual'),
)

# Show the figure
fig.show()

In [41]:
#app = Twitter("session")

all_tweets = app.get_tweets("StephenCurry30")
for tweet in all_tweets:
    print(tweet)

In [39]:
all_tweets

UserTweets(user_id=42562446, count=0)

In [61]:
import plotly.express as px

# Tokenize and count the words using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tweets_noneg['cleaned_text'])
word_counts = X.sum(axis=0)

# Convert word counts to a DataFrame
word_counts_df = pd.DataFrame({'word': vectorizer.get_feature_names(), 'count': word_counts.tolist()[0]})

# Sort the DataFrame by count in descending order
word_counts_df = word_counts_df.sort_values(by='count', ascending=False)

# Select the top 10 words
top_10_words = word_counts_df.head(10)

# Create a bar plot using Plotly
fig = px.bar(
    top_10_words,
    x='count',
    y='word',
    orientation='h',
    title='Top 10 Words Count',
    labels={'count': 'Count', 'word': 'Word'},
    template='plotly_dark'
)

fig.show()

In [83]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# Assuming tweets_noneg is your DataFrame
tweets_noneg['length'] = tweets_noneg.text.str.split().apply(len)

# Create histogram using Plotly Express
histogram = px.histogram(
    tweets_noneg[tweets_noneg['airline_sentiment'] == 'negative'],
    x='length',
    nbins=20,
    labels={'length': 'Text Length'},
    color_discrete_sequence=['red'],
)

# Add histogram trace with outlined bins
histogram.update_traces(
    marker_line_width=1.5,
    marker_line_color='white',
    opacity=0.7,
)

# Create descriptive statistics table
describe = tweets_noneg.length[tweets_noneg.airline_sentiment == 'negative'].describe().to_frame().round(2)
table = go.Figure(data=[go.Table(
    header=dict(values=['Statistic', 'Value']),
    cells=dict(values=[describe.index, describe['length']]),
)])

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Distribution of Text Length', 'Descriptive Statistics'],
    column_widths=[0.7, 0.3],
    specs=[[{'type': 'xy'}, {'type': 'table'}]]
)

# Add histogram to subplot
for trace in histogram.data:
    fig.add_trace(trace, row=1, col=1)

# Add table to subplot
for trace in table.data:
    fig.add_trace(trace, row=1, col=2)

# Update layout for subplot-like appearance
fig.update_layout(
    title='Distribution of Tweet Length and Descriptive Statistics for Negative Sentiment Tweets',
    template='plotly_dark'
)   

# Add explicit x-axis and y-axis labels
fig.update_xaxes(title_text='Length of Tweet by Word Count', row=1, col=1)
fig.update_yaxes(title_text='Count', row=1, col=1)

fig.show()
